# Application of Optuna to find the optimal hyperparameters for transfer learning or fine tuning the pre-trained models

@authors:

Ali Aghababaei, aaghababaei78@gmail.com  and   Roya Arian, royaarian101@gmail.com






This code was used to find best hyperparameters to classify MS and Normal cases using SLO images. However it can be used in any other application.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Enter the file path in your Google Drive

In [ ]:
%cd '/Enter your path'

/content/drive/MyDrive/Slo_classification


### Installing Optuna

In [ ]:
!pip install optuna
!pip install optuna-integration
import optuna
from optuna.trial import TrialState
from optuna.integration import TFKerasPruningCallback

In [ ]:
####################### transfer learning ##############################

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, BatchNormalization, Dropout
import tensorflow as tf
from keras.layers import Input


def classifier (trial):

  # load one of the state-of-the-art models (VGG16, VGG19, Inception-V3, Resnet50, IfitientNet, efficientnet_b0 to b7)
  base_model = tf.keras.applications.VGG19(
        weights='imagenet',
        include_top=False,
        input_shape=(128, 128, 3),
        )


  ################## fine tune
  for layer in base_model.layers:
    layer.trainable = False


  # base_model.trainable = False

  model = Sequential ()

  model.add (base_model)

  model.add (Flatten ())

  dropout_l0 = trial.suggest_float("dropout_l0", 0, 0.7,step=0.1)

  model.add (Dropout (dropout_l0))


  n_layers = trial.suggest_int("n_layers", 1, 5)


  for i in range(n_layers):

      n_units = trial.suggest_int("n_units_l{}".format(i), 8, 5000, log = True)

      model.add (Dense (n_units, activation = 'relu'))

      dropout = trial.suggest_float("dropout_l{}".format (i+1), 0, 0.7,step=0.1)

      model.add (Dropout (rate = dropout))

  model.add (Dense (1, activation = 'sigmoid'))

  lr = trial.suggest_float ('lr', 1e-5, 1e-3, log = True)

  my_optimizer = tf.keras.optimizers.Adam (learning_rate= lr)


  model.compile(optimizer=my_optimizer, loss='binary_crossentropy', metrics='accuracy')


  return model

### Applying subject-wise approach to split validation and train data

To prevent data leakage between train and validation datasets, a “subject-wise” approach was followed that involves putting all images belonging to each
individual, regardless of its left-or-right orientation, in a single group.


### Creating Dataset

Create a dictionary for your dataset in which each key refers to one patient and the value of each key must be a nested-dictionary with own key and value. Keyes in nested dictionary indicate the number of images belonging to the patient and values are the corresponding numpy array of the images. Save this dictionary as a pickle file named (“subjects_slo_data.pkl”). However, the label dictionary contains keys and values, where the keys are the same as the keys in the image dictionary and the values are the patient’s label. Save this dictionary as a pickle file named (“labels_slo_data.pkl”)
For example patient number one has 4 images with label = 1 and patient number two has 2 images with label = 0. Therefore, the corresponding dictionaries are as follow:

•	images [0] is a dictionary with size (4):

•	np.shape(images[0][0])  = (128 ×128 ×3)

•	np.shape(images [0][1]) = (128 ×128 ×3)

•	np.shape(images [0][2]) = (128 ×128 ×3)

•	np.shape(images [0][3]) = (128 ×128 ×3)


	labels_train [0] = 1



•	images [1] is a dictionary with size (2):

•	np.shape(images [1][0]) = (128 ×128 ×3)

•	np.shape(images [1][1]) = (128 ×128 ×3)


	labels_train [1] = 0


Remember to resize your images to (128 × 128 × 3) or change the input_shape in previous cell according to your desired size.


In [ ]:
import keras
import pickle
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold (n_splits = 5, shuffle = True, random_state = None)

def preparing(x, y):

    data  = []
    label = []
    for i in x:
        for j in range(len(x[i])):
            data.append(x[i][j])
            label.append(y[i])

    data = np.reshape(data, np.shape(data))
    return data, label


def objective (trial):

  keras.backend.clear_session()

  # load the data and the coresponding lables pickle file
  images_train = pickle.load(open("subjects_slo_data.pkl", 'rb'))
  labels_train = pickle.load(open("labels_slo_data.pkl", 'rb'))

  train_index, val_index = next (skf.split (images_train, list(labels_train.values())))


  x_train = {i: images_train[i] for i in train_index}

  x_valid = {i: images_train[i] for i in val_index}

  y_train = {i: labels_train[i] for i in train_index}

  y_valid = {i: labels_train[i] for i in val_index}

  x_train,y_train = preparing(x_train,y_train)

  x_valid,y_valid = preparing(x_valid,y_valid)

  # Normalizing
  x_train /= 255
  x_valid /= 255

  batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64, 128])

  datagen = ImageDataGenerator(
  rotation_range= 5, # rotation
  width_shift_range= [-30, 30], # horizontal shift
  height_shift_range= [-5, 5] , # vertical shift
  zoom_range= 0.2,
  vertical_flip= True , # vertical flip
  brightness_range= [0.2, 1.5]
    )


  batch=np.zeros_like(x_train, dtype=np.float32)

  batch_label=np.zeros_like(y_train, dtype=np.float32)

  for i in range(len(x_train)):

    x1= x_train[i,:,:,:].copy()

    x1=x1.reshape((1, ) + x1.shape)

    x = datagen.flow(x1, batch_size=1, seed=42) # to make the result reproducible

    batch[i,:,:,:] = x.next()

    batch_label[i] = y_train[i]

  x_train = np.concatenate([x_train,batch])

  y_train = np.concatenate([y_train,batch_label])

        ####################################################################
        # classification via my model
        ####################################################################


  model = classifier (trial)


  # Generate our trial model.

  model.fit(x_train,
            np.asarray(y_train, dtype=np.float64),
            batch_size= batch_size,
            epochs=50,
            callbacks=[TFKerasPruningCallback(trial, "val_accuracy"), EarlyStopping(patience=10, verbose=1),
        ReduceLROnPlateau(factor=0.1, patience=10, min_lr=1e-6),
        ModelCheckpoint(f'slo.h5', verbose=1, save_best_only=True, save_weights_only=True)],
            validation_data=(x_valid, np.asarray(y_valid, dtype=np.float64)),
            )
  model.load_weights(f'slo.h5')
  score = model.evaluate (x_valid, np.asarray(y_valid, dtype=np.float64), verbose = 1)

  return score [1]

In [ ]:
study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())

study.optimize(objective, n_trials=100)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))